# The purpose of this notebook is to create datasets consisting of Bitcoin prices together with lagged tweets

In [2]:
#imports
import pandas as pd 
from datetime import datetime
from datetime import timedelta
import os

### Load BTC data with dates

In [3]:
btc_data = pd.read_csv("./../datasets/general/BTCDATAwithdate.csv") 

In [4]:
btc_data.head()

,Unnamed: 0,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-31 11:04:00+00:00
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-31 11:05:00+00:00
2,3500002,6949.89,1.264825,8789.841563,6949.454444,2018-08-31 11:06:00+00:00
3,3500003,6950.61,1.948446,13550.262119,6954.393879,2018-08-31 11:07:00+00:00
4,3500004,6949.51,5.503697,38235.502385,6947.240108,2018-08-31 11:08:00+00:00


In [5]:
#Change timestamp to data in bitcoin data
btc_data['timestamp'] = pd.to_datetime(btc_data['timestamp'], utc = True)  

### Create lag

In [6]:
#choose lag
lag = 7
#this can be hours or days
granularity = "days"

In [7]:
#move by lag 
btc_data['timestamp'] = (btc_data['timestamp'] - timedelta(hours=lag)) if granularity == 'hours' else (btc_data['timestamp'] - timedelta(days=lag))

In [8]:
btc_data

,Unnamed: 0,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-24 11:04:00+00:00
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-24 11:05:00+00:00
2,3500002,6949.89,1.264825,8789.841563,6949.454444,2018-08-24 11:06:00+00:00
3,3500003,6950.61,1.948446,13550.262119,6954.393879,2018-08-24 11:07:00+00:00
4,3500004,6949.51,5.503697,38235.502385,6947.240108,2018-08-24 11:08:00+00:00
...,...,...,...,...,...,...
1072252,4572252,10325.89,0.081143,837.754188,10324.419779,2020-09-07 01:56:00+00:00
1072253,4572253,10331.97,0.572817,5918.028741,10331.444396,2020-09-07 01:57:00+00:00
1072254,4572254,10331.47,2.489909,25711.238323,10326.175283,2020-09-07 01:58:00+00:00
1072255,4572255,10334.78,4.572660,47253.747619,10333.972651,2020-09-07 01:59:00+00:00


## According to the analyser used to create polarity scores, load the file

In [9]:
#this can either be TextBlob or vader
# analyser = "TextBlob"
analyser = "vader"
#whether to use cleaned data
cleaned = True
#whether to include polarity
with_polarity = True

folder = "./../datasets/general/with_sentiment/"+analyser if with_polarity else "./../datasets/general"

if(with_polarity and cleaned):
    folder = folder + "/cleaned"
    
print(folder)

#get tweets
tweet_data = pd.read_csv(folder+"/tweets.csv") if with_polarity else pd.read_csv(folder+"/tweets_cleaned.csv")

./../datasets/general/with_sentiment/vader/cleaned


C:\Users\dell\Anaconda3\envs\fyp\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
tweet_data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,index,id,user,fullname,url,timestamp,replies,likes,retweets,text,compound,pos_pol,neg_pol,neu_pol
0,0,0,0,1,1,1.132977e+18,bitcointe,Bitcointe,NaN,2019-05-27 11:49:18+00,0.0,0.0,0.0,Cardano Digitize Currencies EOS 6500 ROI AT T ...,-0.1027,0.000,0.104,0.896
1,1,1,1,2,2,1.132977e+18,3eyedbran,Bran - 3 Eyed Raven,NaN,2019-05-27 11:49:06+00,0.0,2.0,1.0,Another Test tweet that wasnt caught stream bi...,0.0000,0.000,0.000,1.000
2,2,2,2,3,3,1.132977e+18,DetroitCrypto,J. Scardina,NaN,2019-05-27 11:49:22+00,0.0,0.0,0.0,Current Crypto Prices BTC 872199 USD ETH 26662...,0.0000,0.000,0.000,1.000
3,3,3,3,4,4,1.132977e+18,mmursaleen72,Muhammad Mursaleen,NaN,2019-05-27 11:49:23+00,0.0,0.0,0.0,Spiv Nosar Baz BITCOIN Is An Asset NOT A Currency,0.3612,0.217,0.000,0.783
4,4,4,4,6,6,1.132977e+18,evilrobotted,evilrobotted,NaN,2019-05-27 11:49:25+00,0.0,0.0,0.0,USER We have been building on real SV What hav...,-0.4767,0.000,0.171,0.829


In [11]:
#remove tweets with neutral score of 1
if(with_polarity):
    tweet_data = tweet_data[tweet_data['neu_pol'] != 1]

In [12]:
#Round timestamps to nearest minute
tweet_data['timestamp'] = pd.to_datetime(tweet_data['timestamp'], utc = True) 
tweet_data['timestamp'] = tweet_data['timestamp'].apply(lambda dt: dt.round('min'))

In [14]:
#concatinate btc prices and tweets on timestamp
final_data = btc_data.merge(tweet_data, how="left", on="timestamp")

In [15]:
final_data

,Unnamed: 0_x,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,timestamp,Unnamed: 0_y,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,...,fullname,url,replies,likes,retweets,text,compound,pos_pol,neg_pol,neu_pol
0,3500000,6936.26,1.001363,6944.466619,6935.011702,2018-08-24 11:04:00+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3500001,6944.90,10.379585,72022.107341,6938.823172,2018-08-24 11:05:00+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3500002,6949.89,1.264825,8789.841563,6949.454444,2018-08-24 11:06:00+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3500003,6950.61,1.948446,13550.262119,6954.393879,2018-08-24 11:07:00+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3500004,6949.51,5.503697,38235.502385,6947.240108,2018-08-24 11:08:00+00:00,836661.0,846074.0,251.0,2483284.0,...,HODLER,/HODL_ER/status/1032977774624079874,0.0,0.0,0.0,LATEST PRICES FOR TOP 10 CRYPTOCURRENCIES USD ...,0.368,0.064,0.0,0.936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6716149,4572252,10325.89,0.081143,837.754188,10324.419779,2020-09-07 01:56:00+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6716150,4572253,10331.97,0.572817,5918.028741,10331.444396,2020-09-07 01:57:00+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6716151,4572254,10331.47,2.489909,25711.238323,10326.175283,2020-09-07 01:58:00+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6716152,4572255,10334.78,4.572660,47253.747619,10333.972651,2020-09-07 01:59:00+00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#drop rows where close is null
final_data = final_data.dropna(axis=0, subset=['Close'])
#drop unwanted columns
wanted_cols = ['id', 'fullname', 'url', 'timestamp', 'replies', 'likes', 'retweets', 'text', 'Close', 'Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price']

if with_polarity:
    wanted_cols = wanted_cols + ['neg_pol', 'neu_pol', 'pos_pol', 'compound']

final_data = final_data[wanted_cols]


In [17]:
#replace empty text with empty string
final_data['text'] = final_data['text'].fillna('')

In [23]:
final_data

,id,fullname,url,timestamp,replies,likes,retweets,text,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,neg_pol,neu_pol,pos_pol,compound
0,NaN,NaN,NaN,2018-08-24 11:04:00+00:00,NaN,NaN,NaN,,6936.26,1.001363,6944.466619,6935.011702,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,2018-08-24 11:05:00+00:00,NaN,NaN,NaN,,6944.90,10.379585,72022.107341,6938.823172,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,2018-08-24 11:06:00+00:00,NaN,NaN,NaN,,6949.89,1.264825,8789.841563,6949.454444,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2018-08-24 11:07:00+00:00,NaN,NaN,NaN,,6950.61,1.948446,13550.262119,6954.393879,NaN,NaN,NaN,NaN
4,1.032978e+18,HODLER,/HODL_ER/status/1032977774624079874,2018-08-24 11:08:00+00:00,0.0,0.0,0.0,LATEST PRICES FOR TOP 10 CRYPTOCURRENCIES USD ...,6949.51,5.503697,38235.502385,6947.240108,0.0,0.936,0.064,0.368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6716149,NaN,NaN,NaN,2020-09-07 01:56:00+00:00,NaN,NaN,NaN,,10325.89,0.081143,837.754188,10324.419779,NaN,NaN,NaN,NaN
6716150,NaN,NaN,NaN,2020-09-07 01:57:00+00:00,NaN,NaN,NaN,,10331.97,0.572817,5918.028741,10331.444396,NaN,NaN,NaN,NaN
6716151,NaN,NaN,NaN,2020-09-07 01:58:00+00:00,NaN,NaN,NaN,,10331.47,2.489909,25711.238323,10326.175283,NaN,NaN,NaN,NaN
6716152,NaN,NaN,NaN,2020-09-07 01:59:00+00:00,NaN,NaN,NaN,,10334.78,4.572660,47253.747619,10333.972651,NaN,NaN,NaN,NaN


### Save to dataset file

In [20]:
folder = './../datasets/tweets_and_prices/'+analyser if with_polarity else './../datasets/tweets_and_prices/cleaned'

if (cleaned and with_polarity):
    folder = folder + '/cleaned'

if not os.path.exists(folder):
    os.makedirs(folder)

In [22]:
#save file
filename_to_save = folder+'/final_data_lag_'+granularity+'_'+str(lag)+'.csv'
final_data.to_csv(filename_to_save)